In [ ]:
%pip install --upgrade plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 47.4 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.15.0
    Uninstalling plotly-5.15.0:
      Successfully uninstalled plotly-5.15.0


In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
data = pd.read_csv("hvac_sales.csv", index_col=0)
data.head()

,Region,Type,Units,AFUE,SEER,Ducted,Refrigerant,Cooling (Btu/hr),Heating (Btu/hr),MotorType,HSPF,FinalEstimate,EER,FuelType
Year,,,,,,,,,,,,,,
2021,Northeastern,AC,33,NaN,13.00,0.00,410.00,"18,000.00",NaN,NaN,NaN,165.00,NaN,NaN
2021,Northeastern,AC,326,NaN,13.00,0.00,410.00,"24,000.00",NaN,NaN,NaN,"1,631.00",NaN,NaN
2021,Northeastern,AC,289,NaN,13.00,0.00,410.00,"30,000.00",NaN,NaN,NaN,"1,446.00",NaN,NaN
2021,Northeastern,AC,31,NaN,13.00,0.00,410.00,"42,000.00",NaN,NaN,NaN,155.00,NaN,NaN
2021,Northeastern,AC,49,NaN,13.00,0.00,410.00,"48,000.00",NaN,NaN,NaN,245.00,NaN,NaN


In [ ]:
co = data[data["Units"] == -3]
co

,Region,Type,Units,AFUE,SEER,Ducted,Refrigerant,Cooling (Btu/hr),Heating (Btu/hr),MotorType,HSPF,FinalEstimate,EER,FuelType
Year,,,,,,,,,,,,,,
2021,NY,AC,-3,NaN,19.00,0.00,410.00,"30,000.00",NaN,NaN,NaN,-13.00,NaN,NaN


## EDA

In [ ]:
data.shape

(7815, 14)

In [ ]:
data.columns

Index(['Region', 'Type', 'Units', 'AFUE', 'SEER', 'Ducted', 'Refrigerant',
       'Cooling (Btu/hr)', 'Heating (Btu/hr)', 'MotorType', 'HSPF',
       'FinalEstimate', 'EER', 'FuelType'],
      dtype='object')

## Data cleaning

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7815 entries, 2021 to 2017
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Region            7815 non-null   object 
 1   Type              7815 non-null   object 
 2   Units             7815 non-null   int64  
 3   AFUE              2840 non-null   float64
 4   SEER              4776 non-null   float64
 5   Ducted            4857 non-null   float64
 6   Refrigerant       4845 non-null   float64
 7   Cooling (Btu/hr)  4854 non-null   float64
 8   Heating (Btu/hr)  2919 non-null   float64
 9   MotorType         1170 non-null   object 
 10  HSPF              3459 non-null   object 
 11  FinalEstimate     7815 non-null   float64
 12  EER               3274 non-null   float64
 13  FuelType          2852 non-null   object 
dtypes: float64(8), int64(1), object(5)
memory usage: 915.8+ KB


In [ ]:
print(f'Any duplicates? {data.duplicated().values.any()}')

Any duplicates? True


## Descriptive Statistics

In [ ]:
data.describe()

,Units,AFUE,SEER,Ducted,Refrigerant,Cooling (Btu/hr),Heating (Btu/hr),FinalEstimate,EER
count,"7,815.00","2,840.00","4,776.00","4,857.00","4,845.00","4,854.00","2,919.00","7,815.00","3,274.00"
mean,80.38,88.25,17.56,0.44,407.44,"29,686.03","96,628.98",462.25,11.81
std,273.36,5.98,3.30,0.50,31.43,"14,474.87","36,929.94","1,674.77",1.64
min,-3.00,80.00,13.00,0.00,22.00,"6,000.00","20,000.00",-17.61,9.00
25%,2.00,83.00,15.00,0.00,410.00,"18,000.00","60,000.00",11.71,11.00
50%,9.00,86.00,17.00,0.00,410.00,"24,000.00","100,000.00",45.28,12.00
75%,40.00,95.00,20.00,1.00,410.00,"36,000.00","120,000.00",219.62,13.00
max,"5,299.00",99.00,26.00,1.00,410.00,"72,000.00","160,000.00","49,739.66",20.00


## Visualization

### Sales by Year

In [ ]:
sales_by_year = data.groupby("Year")["Units"].sum()
sales_by_year.head()

Year
2017    116056
2018    138297
2019    136458
2020    108060
2021    129324
Name: Units, dtype: int64

In [ ]:
fig = px.bar(x=sales_by_year.index, y=sales_by_year.values,
            color=sales_by_year.values,
            color_continuous_scale=px.colors.sequential.Sunsetdark,
            title="HVACs sales by Year - NY and Northeastern region (USA)", labels={"color" : "No. of HVACs sold"}, text_auto=True)

fig.update_layout(xaxis_title="Years", yaxis_title="No. of HVACs sold")
fig.show()

### Sales by region

In [ ]:
sales_by_region = data.groupby("Region")["Units"].sum()
sales_by_region.head()

Region
NY              318245
Northeastern    309950
Name: Units, dtype: int64

In [ ]:
fig = px.bar(x=sales_by_region.index, y=sales_by_region.values,
            color=sales_by_region.values,
            color_continuous_scale=px.colors.sequential.Agsunset, title="HVACs sales by Region - USA (2017-2021)", labels={"color" : "No. of HVACs sold"}, text_auto=True)

fig.update_layout(xaxis_title="Region", yaxis_title="No. of HVACs sold")
fig.show()

### HVAC type distribution

In [ ]:
sales_by_type = data.groupby("Type")["Units"].sum()
sales_by_type.head()

Type
AC         239155
ASHP       159909
Boiler      52481
Furnace    171673
GEU          4977
Name: Units, dtype: int64

In [ ]:
fig = px.pie(labels = sales_by_type.index, values=sales_by_type.values, title="HVACs sales by Type - NY and Northeastern region (2017-2021)", names=sales_by_type.index, hole=0.4)
fig.update_traces(textposition="inside", textfont_size=15, textinfo="label+percent+value")
fig.show()